# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
M. Sharma  ->  Y. M. Sharma  |  ['M. Sharma']
K. Lee  ->  K. Lee  |  ['K. Lee']
T. Pfeil  ->  T. Pfeil  |  ['T. Pfeil']
H. Klahr  ->  H. Klahr  |  ['H. Klahr']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
T. Mikal-Evans  ->  T. Mikal-Evans  |  ['T. Mikal-Evans']
Arxiv has 79 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2310.06887


extracting tarball to tmp_2310.06887...

 done.


A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']


...

extracting tarball to tmp_2310.07358...

 done.
Retrieving document from  https://arxiv.org/e-print/2310.07714


extracting tarball to tmp_2310.07714... done.


L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
T. Mikal-Evans  ->  T. Mikal-Evans  |  ['T. Mikal-Evans']


Found 55 bibliographic references in tmp_2310.07714/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.06887-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.06887) | **FRESCO: An extended, massive, rapidly rotating galaxy at z=5.3**  |
|| E. J. Nelson, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2023-10-12*|
|*Comments*| *Fig. 3 shows the main result*|
|**Abstract**| With the remarkable sensitivity and resolution of JWST in the infrared, measuring rest-optical kinematics of galaxies at $z>5$ has become possible for the first time. This study pilots a new method for measuring galaxy dynamics for highly multiplexed, unbiased samples by combining FRESCO NIRCam grism spectroscopy and JADES medium-band imaging. Here we present one of the first JWST kinematic measurements for a galaxy at $z>5$. We find a significant velocity gradient, which, if interpreted as rotation yields $V_{rot} = 240\pm50$km/s and we hence refer to this galaxy as Twister-z5. With a rest-frame optical effective radius of $r_e=2.25$kpc, the high rotation velocity in this galaxy is not due to a compact size as may be expected in the early universe but rather a high total mass, ${\rm log(M}_{dyn}/{\rm M}_\odot)=11.0\pm0.2$. This is a factor of roughly 4x higher than the stellar mass within the effective radius. We also observe that the radial H$\alpha$ equivalent width profile and the specific star formation rate map from resolved stellar population modeling is centrally depressed by a factor of $\sim1.5$ from the center to $r_e$. Combined with the morphology of the line-emitting gas in comparison to the continuum, this centrally suppressed star formation is consistent with a star-forming disk surrounding a bulge growing inside-out. While large, rapidly rotating disks are common to z~2, the existence of one after only 1Gyr of cosmic time, shown for the first time in ionized gas, adds to the growing evidence that some galaxies matured earlier than expected in the history of the universe. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.07332-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.07332) | **Dust Coagulation Reconciles Protoplanetary Disk Observations with the  Vertical Shear Instability -- Part I: Dust Coagulation and the VSI Dead Zone**  |
|| <mark>T. Pfeil</mark>, T. Birnstiel, <mark>H. Klahr</mark> |
|*Appeared on*| *2023-10-12*|
|*Comments*| *27 pages, 15 figures, Accepted for publication in The Astrophysical Journal*|
|**Abstract**| Protoplanetary disks exhibit a vertical gradient in angular momentum, rendering them susceptible to the Vertical Shear Instability (VSI). The most important condition for the onset of this mechanism is a short timescale of thermal relaxation ($\lesssim 0.1$ orbital timescales). Simulations of fully VSI active disks are characterized by turbulent, vertically extended dust layers. This is in contradiction with recent observations of the outer regions of some protoplanetary disks, which appear highly settled. In this work, we demonstrate that the process of dust coagulation can diminish the cooling rate of the gas in the outer disk and extinct the VSI activity. Our findings indicate that the turbulence strength is especially susceptible to variations in the fragmentation velocity of the grains. A small fragmentation velocity of $\approx 100 \mathrm{\, cm \,s^{-1}}$ results in a fully turbulent simulation, whereas a value of $\approx 400 \mathrm{\, cm \,s^{-1}}$ results in a laminar outer disk, being consistent with observations. We show that VSI turbulence remains relatively unaffected by variations in the maximum particle size in the inner disk regions. However, we find that dust coagulation can significantly suppress the occurrence of VSI turbulence at larger distances from the central star. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.07714-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.07714) | **Clouds and Clarity: Revisiting Atmospheric Feature Trends in  Neptune-size Exoplanets**  |
|| J. Brande, et al. -- incl., <mark>L. Kreidberg</mark>, <mark>T. Mikal-Evans</mark> |
|*Appeared on*| *2023-10-12*|
|*Comments*| *Submitted to ApJL. 11 pages, 3 figures, 4 tables*|
|**Abstract**| Over the last decade, precise exoplanet transmission spectroscopy has revealed the atmospheres of dozens of exoplanets, driven largely by observatories like the \textit{Hubble} Space Telescope. One major discovery has been the ubiquity of atmospheric aerosols, often blocking access to exoplanet chemical inventories. Tentative trends have been identified, showing that the clarity of planetary atmospheres may depend on equilibrium temperature. Previous work has often grouped dissimilar planets together in order to increase the statistical power of any trends, but it remains unclear from observed transmission spectra whether these planets exhibit the same atmospheric physics and chemistry. We present a re-analysis of a smaller, more physically similar sample of 15 exo-Neptune transmission spectra across a wide range of temperatures (200-1000 K). Using condensation cloud and hydrocarbon haze models, we find that the exo-Neptune population is best described by very low cloud sedimentation efficiency ($\mathrm{f_{sed}}\sim0.01$) and high metallicity ($100\times$ Solar). There is an intrinsic astrophysical scatter of $\sim0.5$ scale height, perhaps evidence of stochasticity in these planets' formation processes. Observers should expect significant attenuation in transmission spectra of Neptune-size exoplanets, up to 6 scale heights for equilibrium temperatures between 500 and 800~K. With JWST's greater wavelength sensitivity, colder ($<500$~K) planets should be high-priority targets given their comparative rarity, clearer atmospheres, and the need to distinguish between the ``super-puffs'' and more typical gas-dominated planets. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.06898-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.06898) | **The origin of supermassive black holes at cosmic dawn**  |
|| R. Sharma, <mark>M. Sharma</mark> |
|*Appeared on*| *2023-10-12*|
|*Comments*| *9 pages, 6 figures, submitted to MNRAS, comments welcome*|
|**Abstract**| We investigate the steady spherically symmetric accretion in the combined potential of a central black hole and a dark matter halo. For the halo, we consider a Hernquist and an NFW potential and calculate the critical points of the flow. We find that the trans-sonic solution to the centre is not possible without a black hole, whereas two types of trans-sonic solutions are possible in its presence. We also derive the mass accretion rate for a black hole at the centre of a dark matter halo. Our results indicate two phases of accretion. The first is an initial phase with a low accretion rate that depends on the black hole mass, followed by a second phase with a high accretion rate that depends on the halo mass. In the second phase, the black hole mass increases rapidly to supermassive scales, which explains the existence of quasars at redshift $z\ge 6$ and also the recently detected supermassive black hole (SMBH) by the James Webb Space Telescope (JWST). Further, we calculate the evolution of the Eddington ratio for growing black holes. The accretion is mostly sub-Eddington except for a short super-Eddington episode when the mass accretion rate transitions from low to high. However, during that episode, the black hole mass is likely inadequate to hinder accretion through radiative feedback. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.07358-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.07358) | **CMB delensing with deep learning**  |
|| S. Ni, Y. Li, <mark>X. Zhang</mark> |
|*Appeared on*| *2023-10-12*|
|*Comments*| *10 pages, 10 figures*|
|**Abstract**| The cosmic microwave background (CMB) provides an important source for the study of weak gravitational lensing. The lensed CMB can be used to constrain cosmological parameters, but it also has smoothing effect on the original CMB. The angular power spectrum of the unlensed CMB has sharper acoustic peaks and more prominent damping tails, allowing for improved inferences of cosmological parameters affecting these characteristics. Although delensing reduces the B-mode power spectrum, it aids the search for primordial gravitational waves and allows for lower variance reconstruction of lensing and other sources of secondary CMB anisotropies. In this work, we investigate whether the deep learning methods, concretely the U-Net++ algorithm here, can play a crucial role in CMB delensing. We consider three fields, i.e., T, Q and U sky maps, present the angular power spectra of CMB delensed TT, EE and BB, and compare them with the unlensed CMB angular power spectra. The results show that the angular power spectrum of the lensed CMB processed by U-Net++ is in great agreement with the unlensed CMB angular power spectrum. Therefore, the U-Net++ based CMB delensing can efficiently eliminate the effects caused by weak gravitational lensing and shed new light on the CMB power spectrum for future CMB experiments. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.07163-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.07163) | **The Qitai Radio Telescope**  |
|| N. Wang, et al. -- incl., <mark>K. Lee</mark> |
|*Appeared on*| *2023-10-12*|
|*Comments*| *12 pages, 11 figures, accepted for publication in Science China Physics, Mechanics & Astronomy*|
|**Abstract**| This study presents a general outline of the Qitai radio telescope (QTT) project. Qitai, the site of the telescope, is a county of Xinjiang Uygur Autonomous Region of China, located in the east Tianshan Mountains at an elevation of about 1800 m. The QTT is a fully steerable, Gregorian type telescope with a standard parabolic main reflector of 110 m diameter. The QTT has adopted an um-brella support, homology-symmetric lightweight design. The main reflector is active so that the deformation caused by gravity can be corrected. The structural design aims to ultimately allow high-sensitivity observations from 150 MHz up to 115 GHz. To satisfy the requirements for early scientific goals, the QTT will be equipped with ultra-wideband receivers and large field-of-view mul-ti-beam receivers. A multi-function signal-processing system based on RFSoC and GPU processor chips will be developed. These will enable the QTT to operate in pulsar, spectral line, continuum and Very Long Baseline Interferometer (VLBI) observing modes. Electromagnetic compatibility (EMC) and radio frequency interference (RFI) control techniques are adopted throughout the system design. The QTT will form a world-class observational platform for the detection of low-frequency (nanoHertz) gravitational waves through pulsar timing array (PTA) techniques, pulsar surveys, the discovery of binary black-hole systems, and exploring dark matter and the origin of life in the universe. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2310.06887.md
    + _build/html/tmp_2310.06887/./figs_v2/sfms_v2.png
    + _build/html/tmp_2310.06887/./figs_v2/mb_hamap.png
    + _build/html/tmp_2310.06887/./figs_v2/rot_ims_21jul23.png
    + _build/html/tmp_2310.06887/./figs_v2/rotcurves_21jul23.png
exported in  _build/html/2310.07332.md
    + _build/html/tmp_2310.07332/./radmcCompareAlphaAndVSI200_AnaModel_Linear.png
    + _build/html/tmp_2310.07332/./radmcCompareAlphaAndVSI400_AnaModel_Linear.png
    + _build/html/tmp_2310.07332/./radmcCompareAlphaAndVSI100_AnaModel_Linear.png
exported in  _build/html/2310.07714.md
    + _build/html/tmp_2310.07714/./combined_comparison_3amu.png
    + _build/html/tmp_2310.07714/./rainbow_stack_3amu.png
    + _build/html/tmp_2310.07714/./trend_sample.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\ha}{H\alpha\xspace}$
$\newcommand{\mstar}{M_*\xspace}$
$\newcommand{\msun}{M_\odot\xspace}$
$\newcommand{\twister}{Twister-z5}$</div>



<div id="title">

# FRESCO: An extended, massive, rapidly rotating galaxy at $z=5.3$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.06887-b31b1b.svg)](https://arxiv.org/abs/2310.06887)<mark>Appeared on: 2023-10-12</mark> -  _Fig. 3 shows the main result_

</div>
<div id="authors">

E. Nelson, et al. -- incl., <mark>A. d. Graaff</mark>

</div>
<div id="abstract">

**Abstract:** With the remarkable sensitivity and resolution of JWST in the infrared, measuring rest-optical kinematics of galaxies at $z>5$ has become possible for the first time. This study pilots a new method for measuring galaxy dynamics for highly multiplexed, unbiased samples by combining FRESCO NIRCam grism spectroscopy and JADES medium-band imaging. Here we present one of the first JWST kinematic measurements for a galaxy at $z>5$ . We find a significant velocity gradient, which, if interpreted as rotation yields $V_{rot} = 240\pm50$ km/s and we hence refer to this galaxy as Twister-z5. With a rest-frame optical effective radius of $r_e=2.25$ kpc, the high rotation velocity in this galaxy is not due to a compact size as may be expected in the early universe but rather a high total mass, ${\rm log(M}_{dyn}/{\rm M}_\odot)=11.0\pm0.2$ . This is a factor of roughly 4 $\times$ higher than the stellar mass within $r_e$ . We also observe that the radial H $\alpha$ equivalent width profile and the specific star formation rate map from resolved stellar population modeling is centrally depressed by a factor of $\sim1.5$ from the center to $r_e$ . Combined with the morphology of the line-emitting gas in comparison to the continuum, this centrally suppressed star formation is consistent with a star-forming disk surrounding a bulge growing inside-out. While large, rapidly rotating disks are common to $z\sim2$ , the existence of one after only 1Gyr of cosmic time, shown for the first time in ionized gas, adds to the growing evidence that some galaxies matured earlier than expected in the history of the universe.

</div>

<div id="div_fig1">

<img src="tmp_2310.06887/./figs_v2/sfms_v2.png" alt="Fig3" width="100%"/>

**Figure 3. -** Star-forming main sequence for galaxies with grism spectroscopic redshifts $4.5<z<6$ in FRESCO with detected $\ha$. Star formation rates are computed from the measured $\ha$ fluxes corrected for dust attenuation using empirical relations based on the UV slope \citep{shivaei:20}. A fit to the SFMS from \cite{speagle:14} is indicated by the red line for context. The typical measurement error is indicated by the cross in the lower right corner. $\twister$, the galaxy featured in the present study, is at the very massive end of this distribution on the upper end of a continuation of the locus of points. (*fig:sfms*)

</div>
<div id="div_fig2">

<img src="tmp_2310.06887/./figs_v2/mb_hamap.png" alt="Fig6" width="100%"/>

**Figure 6. -** We infer the spatial distribution of $\ha$(right) from the difference between the F444W broadband filter (center) and F410M medium band filter (left) which covers the $\ha$ emission line at this redshift. The circle in the middle panel shows the size (FWHM) of the PSF. (*fig:mbha*)

</div>
<div id="div_fig3">

<img src="tmp_2310.06887/./figs_v2/rot_ims_21jul23.png" alt="Fig7.1" width="50%"/><img src="tmp_2310.06887/./figs_v2/rotcurves_21jul23.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** Velocity gradient.  As  described  in \S\ref{kinematics}, we fit the light weighted center along the x-axis in the grism image, which corresponds to the dispersion direction (left), F444W direct image (center), and a stack of the rest-frame UV emission (right).
    The grism has a spectral resolution of $R\sim1600$, meaning that the grism spectrum contains both spatial and spectral information across in the dispersion direction. The difference between the grism and direct image centroids is the velocity in that spatial pixel. The error bars on the velocity curve are given by the difference between using the rest-UV emission and F444W direct image. (*fig:kinmodel*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.06887"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\expnum}[1]{\num[scientific-notation=true, exponent-product=\times, round-mode=places, round-precision=2]{#1}}$
$\newcommand{\rev}[1]{{#1}}$
$\newcommand{\hubert}[1]{{#1}}$
$\newcommand{\report}[1]{{{#1}}}$
$\newcommand{\reporttwo}[1]{{{#1}}}$
$\newcommand{\rhog}{\rho_{\mathrm{g}}}$
$\newcommand{\rhod}{\rho_{\mathrm{d}}}$
$\newcommand{\vg}{\vec{v}_{\mathrm{g}}}$
$\newcommand{\vd}{\vec{v}_{\mathrm{d}}}$
$\newcommand{\amax}{a_{\mathrm{max}}}$
$\newcommand{\amin}{a_{\mathrm{min}}}$
$\newcommand{\aint}{a_{\mathrm{int}}}$
$\newcommand{\adr}{a_{\mathrm{dr}}}$
$\newcommand{\vdr}{v_{\mathrm{dr}}}$
$\newcommand{\tfr}{t_\mathrm{fric}}$
$\newcommand{\St}{\mathrm{St}}$
$\newcommand{\Rey}{\mathfrak{Re}}$
$\newcommand{\OmK}{\Omega_\mathrm{K}}$
$\newcommand{\der}[2]{\frac{\partial{#1}}{\partial{#2}}}$
$\newcommand{\hd}{h_{\mathrm{d}}}$
$\newcommand{\Mp}{m_{\mathrm{p}}}$
$\newcommand{\Sd}{\Sigma_{\mathrm{d}}}$
$\newcommand{\planck}{\kappa_{\mathrm{P}}}$
$\newcommand{\dpy}{{\normalfont\texttt{DustPy}}}$
$\newcommand{\pluto}{{\normalfont\texttt{PLUTO}}}$
$\newcommand{\radmc}{{\normalfont\texttt{RADMC-3D}}}$
$\newcommand{\tpop}{{\normalfont\texttt{2pop}}}$
$\newcommand{\noref}{{\textbf{\color{red}!!!}}}$
$\newcommand{\equationautorefname}{Equation}$
$\newcommand{\figureautorefname}{Figure}$
$\newcommand{\subsectionautorefname}{Section}$
$\newcommand{\sectionautorefname}{Section}$
$\newcommand{\tableautorefname}{Table}$</div>



<div id="title">

# Dust Coagulation Reconciles Protoplanetary Disk Observations with the Vertical Shear Instability $\$$\footnotesize$ Part I: Dust Coagulation and the VSI Dead Zone

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.07332-b31b1b.svg)](https://arxiv.org/abs/2310.07332)<mark>Appeared on: 2023-10-12</mark> -  _27 pages, 15 figures, Accepted for publication in The Astrophysical Journal_

</div>
<div id="authors">

<mark>T. Pfeil</mark>, T. Birnstiel, <mark>H. Klahr</mark>

</div>
<div id="abstract">

**Abstract:** Protoplanetary disks exhibit a vertical gradient in angular momentum, rendering them susceptible to the Vertical Shear Instability (VSI). The most important condition for the onset of this mechanism is a short timescale of thermal relaxation ( $\lesssim 0.1$ orbital timescales). Simulations of fully VSI active disks are characterized by turbulent, vertically extended dust layers. This is in contradiction with recent observations of the outer regions of some protoplanetary disks, which appear highly settled.In this work, we demonstrate that the process of dust coagulation can diminish the cooling rate of the gas in the outer disk and extinct the VSI activity.Our findings indicate that the turbulence strength is especially susceptible to variations in the fragmentation velocity of the grains.A small fragmentation velocity of $\report{$\approx$}$ $\SI{100}{\centi \meter \per \second}$ results in a fully turbulent simulation, whereas a value of $\report{$\approx$}$ $\SI{400}{\centi \meter \per \second}$ results in a laminar outer disk, being consistent with observations.We show that VSI turbulence remains relatively unaffected by variations in the maximum particle size in the inner disk regions. However, we find that dust coagulation can significantly suppress the occurrence of VSI turbulence at larger distances from the central star.

</div>

<div id="div_fig1">

<img src="tmp_2310.07332/./radmcCompareAlphaAndVSI200_AnaModel_Linear.png" alt="Fig7" width="100%"/>

**Figure 7. -** Upper row a: $\radmc$  intensity maps of our VSI simulation with $v_\mathrm{fr}=\SI{200}{\centi \meter \per \second}$ and $\alpha=10^{-3}$, seen edge-on.
    Rows b, c, and d show intensity maps calculated from analytic dust distribution that assume different diffusivities $\delta$. The grain sizes are identical in all simulations.
    We convolve the images with a typical ALMA beam with FWHM of \SI{35}{mas} for a distance of \SI{100}{pc} shown as a grey circle.
    Hatched areas mark regions that have optical depth $\tau\geq 1$. Horizontal hatches correspond to areas for which the $\tau=1$ surface lies on the far side of the disk.
    Diagonally hatched regions mark $\tau=1$ surfaces that lie on the observer's side of the disk.
    The panels on the right-hand side show minor axis cuts through the images along the vertical lines in the intensity maps. $\report${Purple lines in all plots are the minor axis cuts from the VSI simulation (panel a).} (*fig:radmc200*)

</div>
<div id="div_fig2">

<img src="tmp_2310.07332/./radmcCompareAlphaAndVSI400_AnaModel_Linear.png" alt="Fig8" width="100%"/>

**Figure 8. -** Upper row a: $\radmc$  intensity maps of our VSI simulation with $v_\mathrm{fr}=\SI{400}{\centi \meter \per \second}$ and $\alpha=10^{-3}$, seen edge-on.
    Rows b, c, and d show intensity maps calculated from analytic dust distribution that assume different diffusivities $\delta$. The grain sizes are identical in all simulations.
    We convolve the images with a typical ALMA beam with FWHM of \SI{35}{mas} for a distance of \SI{100}{pc} shown as a grey circle.
    Hatched areas mark regions that have optical depth $\tau\geq 1$. Horizontal hatches correspond to areas for which the $\tau=1$ surface lies on the far side of the disk.
    Diagonally hatched regions mark $\tau=1$ surfaces that lie on the observer's side of the disk.
    The panels on the right-hand side show minor axis cuts through the images along the vertical lines in the intensity maps. $\report${Purple lines in all plots are the minor axis cuts from the VSI simulation (panel a).} (*fig:radmc400*)

</div>
<div id="div_fig3">

<img src="tmp_2310.07332/./radmcCompareAlphaAndVSI100_AnaModel_Linear.png" alt="Fig6" width="100%"/>

**Figure 6. -** Upper row a: $\radmc$  intensity maps of our VSI simulation with $v_\mathrm{fr}=\SI{100}{\centi \meter \per \second}$ and $\alpha=10^{-3}$, seen edge-on.
    Rows b, c, and d show intensity maps calculated from analytic dust distribution that assume different diffusivities $\delta$. The grain sizes are identical in all simulations.
    We convolve the images with a typical ALMA beam with FWHM of \SI{35}{mas} for a distance of \SI{100}{pc} shown as a grey circle.
    Hatched areas mark regions that have optical depth $\tau\geq 1$. Horizontal hatches correspond to areas for which the $\tau=1$ surface lies on the far side of the disk.
    Diagonally hatched regions mark $\tau=1$ surfaces that lie on the observer's side of the disk.
    The panels on the right-hand side show minor axis cuts through the images along the vertical lines in the intensity maps. $\report${Purple lines in all plots are the minor axis cuts from the VSI simulation (panel a).} (*fig:radmc100*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.07332"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\hst}{\textit{HST}}$
$\newcommand{\jwst}{\textit{JWST}}$
$\newcommand{\tess}{\textit{TESS}}$
$\newcommand{\rearth}{R_\oplus}$
$\newcommand{\mearth}{M_\oplus}$
$\newcommand{\rsun}{R_\odot}$
$\newcommand{\msun}{M_\odot}$
$\newcommand{\um}{\mum}$
$\newcommand{\ah}{A_{H}}$
$\newcommand{\water}{\mathrm{H}_2\mathrm{O}}$
$\newcommand{\methane}{\mathrm{CH}_4}$
$\newcommand{\dioxide}{\mathrm{CO}_2}$
$\newcommand{\ammonia}{\mathrm{NH}_3}$</div>



<div id="title">

# Clouds and Clarity: Revisiting Atmospheric Feature Trends in Neptune-size Exoplanets

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.07714-b31b1b.svg)](https://arxiv.org/abs/2310.07714)<mark>Appeared on: 2023-10-12</mark> -  _Submitted to ApJL. 11 pages, 3 figures, 4 tables_

</div>
<div id="authors">

J. Brande, et al. -- incl., <mark>L. Kreidberg</mark>, <mark>T. Mikal-Evans</mark>

</div>
<div id="abstract">

**Abstract:** Over the last decade, precise exoplanet transmission spectroscopy has revealed the atmospheres of dozens of exoplanets, driven largely by observatories like the _Hubble_ Space Telescope. One major discovery has been the ubiquity of atmospheric aerosols, often blocking access to exoplanet chemical inventories. Tentative trends have been identified, showing that the clarity of planetary atmospheres may depend on equilibrium temperature. Previous work has often grouped dissimilar planets together in order to increase the statistical power of any trends, but it remains unclear from observed transmission spectra whether these planets exhibit the same atmospheric physics and chemistry. We present a re-analysis of a smaller, more physically similar sample of 15 exo-Neptune transmission spectra across a wide range of temperatures (200-1000 K). Using condensation cloud and hydrocarbon haze models, we find that the exo-Neptune population is best described by very low cloud sedimentation efficiency ( $\mathrm{f_{sed}}\sim0.01$ ) and high metallicity ( $100\times$ Solar). There is an intrinsic astrophysical scatter of $\sim0.5$ scale height, perhaps evidence of stochasticity in these planets’ formation processes. Observers should expect significant attenuation in transmission spectra of Neptune-size exoplanets, up to 6 scale heights for equilibrium temperatures between 500 and 800 K. With $\jwst$ ’s greater wavelength sensitivity, colder ( $<500$ K) planets should be high-priority targets given their comparative rarity, clearer atmospheres, and the need to distinguish between the "super-puffs’’ and more typical gas-dominated planets.

</div>

<div id="div_fig1">

<img src="tmp_2310.07714/./combined_comparison_3amu.png" alt="Fig3" width="100%"/>

**Figure 3. -** Retrieved spectral feature amplitude, $\ah$ , compared to clear, cloudy, and hazy atmosphere models from [Morley, Fortney and Marley (2015)](). For the clear and cloudy models, the dotted lines indicate a $100\times$ Solar metallicity atmosphere, and the dot-dashed lines a $300\times$ Solar atmosphere. All hazy models were calculated at $50\times$ Solar metallicity, the dotted lines are 1\% haze precursor conversion, and the dashed lines are 10\%. The green open markers, shown for context, are the measured $\methane$ -retrieved $\ah$  values for K2-18 b and TOI-270 d. The black open markers, shown for context, are the Kepler-51 planets, which are excluded from the trend analysis due to their exceptionally low densities, and GJ 1214 b, which was found to significantly bias the trend fit due to the extremely precise flatness of its spectrum. (*fig:clear_compare*)

</div>
<div id="div_fig2">

<img src="tmp_2310.07714/./rainbow_stack_3amu.png" alt="Fig2" width="100%"/>

**Figure 2. -** All $\hst$ /WFC3 G141 transmission spectra of our exo-Neptunes, scaled to show the relative $\ah$  values. The retrieved models are colored by the planetary equilibrium temperature, and the spectra are ordered by $\ah$ . (*fig:spectra_stack*)

</div>
<div id="div_fig3">

<img src="tmp_2310.07714/./trend_sample.png" alt="Fig1" width="100%"/>

**Figure 1. -** This work's sample overlaid with known transiting planets. Hexagons indicate planets in our sample also being observed by $\jwst$  through Cycle 2, while triangles are not yet scheduled or approved for $\jwst$  observations. Selected targets have been labeled. (*fig:my_label*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.07714"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

298  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

12  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

9  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
